# Panle Regression - Firm Characteristics

### Random Effects Panel Regression

In [12]:
import pandas as pd
import numpy as np
import datetime as dt
import sklearn
import statsmodels.api as sm
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from linearmodels.panel import PanelOLS
from linearmodels.panel import RandomEffects

Data

In [13]:
df = pd.read_csv("Dataframes/analysts_regression.csv")
df["Date"] = pd.to_datetime(df["Date"])
df

,Instrument,Date,GICS Industry Group Name,Earnings Per Share - Actual Surprise,Earnings Per Share - Actual Surprise AbsVals,Earnings Per Share – Coefficient of Variation,Number of Analysts,Recommendation - Mean (1-5),Recommendation change,Recommendation - Mean (1-5).1,Price Target - Standard Deviation
0,AVY.N,2013-01-01,Materials,11.178,11.178,3.625620,7.0,3.00000,0.00000,3.000,2.44949
1,AVY.N,2013-04-01,Materials,2.482,2.482,2.046169,7.0,2.87500,-0.12500,3.000,3.92272
2,AVY.N,2013-07-01,Materials,1.068,1.068,1.551601,8.0,2.87500,0.00000,2.875,4.30946
3,AVY.N,2013-10-01,Materials,8.095,8.095,2.775994,7.0,2.62500,-0.25000,2.875,3.45230
4,AVY.N,2014-01-01,Materials,1.471,1.471,3.204412,8.0,2.33333,-0.29167,2.625,3.42555
...,...,...,...,...,...,...,...,...,...,...,...
19196,POOL.OQ,2021-10-01,Retailing,17.194,17.194,2.836295,8.0,2.30000,0.00000,2.300,50.48823
19197,POOL.OQ,2022-01-01,Retailing,40.267,40.267,3.269867,8.0,2.30000,0.00000,2.300,41.33833
19198,POOL.OQ,2022-04-01,Retailing,34.342,34.342,6.630736,9.0,2.00000,-0.30000,2.300,37.49206
19199,POOL.OQ,2022-07-01,Retailing,1.503,1.503,3.625249,9.0,2.20000,0.20000,2.000,57.57373


Standardising

In [14]:
rescale = df.copy()
#rescale["Earnings Per Share - Actual Surprise"] = MinMaxScaler().fit_transform(np.array(rescale["Earnings Per Share - Actual Surprise"]).reshape(-1,1))
rescale["Earnings Per Share – Coefficient of Variation"] = MinMaxScaler().fit_transform(np.array(rescale["Earnings Per Share – Coefficient of Variation"]).reshape(-1,1))
rescale["Number of Analysts"] = MinMaxScaler().fit_transform(np.array(rescale["Number of Analysts"]).reshape(-1,1))
rescale["Recommendation - Mean (1-5)"] = MinMaxScaler().fit_transform(np.array(rescale["Recommendation - Mean (1-5)"]).reshape(-1,1))
rescale["Recommendation change"] = MinMaxScaler().fit_transform(np.array(rescale["Recommendation change"]).reshape(-1,1))
rescale["Recommendation - Mean (1-5).1"] = MinMaxScaler().fit_transform(np.array(rescale["Recommendation - Mean (1-5).1"]).reshape(-1,1))
rescale["Price Target - Standard Deviation"] = MinMaxScaler().fit_transform(np.array(rescale["Price Target - Standard Deviation"]).reshape(-1,1))
df_clean = rescale
df_clean

,Instrument,Date,GICS Industry Group Name,Earnings Per Share - Actual Surprise,Earnings Per Share - Actual Surprise AbsVals,Earnings Per Share – Coefficient of Variation,Number of Analysts,Recommendation - Mean (1-5),Recommendation change,Recommendation - Mean (1-5).1,Price Target - Standard Deviation
0,AVY.N,2013-01-01,Materials,11.178,11.178,0.385569,0.125000,0.742857,0.538462,0.666667,0.002079
1,AVY.N,2013-04-01,Materials,2.482,2.482,0.384977,0.125000,0.696428,0.480770,0.666667,0.003330
2,AVY.N,2013-07-01,Materials,1.068,1.068,0.384792,0.145833,0.696428,0.538462,0.625000,0.003658
3,AVY.N,2013-10-01,Materials,8.095,8.095,0.385251,0.125000,0.603571,0.423078,0.625000,0.002930
4,AVY.N,2014-01-01,Materials,1.471,1.471,0.385411,0.145833,0.495236,0.403846,0.541667,0.002908
...,...,...,...,...,...,...,...,...,...,...,...
19196,POOL.OQ,2021-10-01,Retailing,17.194,17.194,0.385273,0.145833,0.482857,0.538462,0.433333,0.042855
19197,POOL.OQ,2022-01-01,Retailing,40.267,40.267,0.385436,0.145833,0.482857,0.538462,0.433333,0.035088
19198,POOL.OQ,2022-04-01,Retailing,34.342,34.342,0.386695,0.166667,0.371428,0.400001,0.433333,0.031824
19199,POOL.OQ,2022-07-01,Retailing,1.503,1.503,0.385569,0.166667,0.445714,0.630770,0.333333,0.048869


In [15]:
df_panel = df_clean.copy()

#### Fixed Effects

In a fixed effects panel regression, the individual-specific effects are modeled as fixed variables that do not vary across time. This means that the coefficients of the independent variables are estimated based on the within-entity variation in the data, which eliminates the effect of time-invariant unobserved heterogeneity.

Fixed effects models are useful when there are time-invariant unobserved variables that may affect the dependent variable, but are not included in the model. By modeling the individual-specific effects as fixed variables, fixed effects models can control for this unobserved heterogeneity and estimate the coefficients of the independent variables based on the within-entity variation, which provides more efficient estimates of the coefficients.

One limitation of fixed effects models is that they do not allow for testing the effect of time-invariant variables on the dependent variable. In addition, fixed effects models may suffer from the incidental parameter problem, which may lead to biased estimates of the coefficients of the independent variables in the presence of a large number of fixed effects.

In [16]:
#reformatting indices of dataframe for panel regression
df_clean['Instrument'] = df_clean['Instrument'].astype('category')
df_clean['Date'] = pd.to_datetime(df_clean['Date'])

# set the index to be the time variable and the cross-sectional variable
df_clean.set_index(['Instrument', 'Date'], inplace=True)
#df_clean

FE Panel Regression with **Absolute** Surprise Values

In [17]:
y = df_clean.loc[:, "Earnings Per Share - Actual Surprise AbsVals"]
X = df_clean.loc[:, "Earnings Per Share – Coefficient of Variation":]

# perform the fixed effects panel regression
fixed_effects_model = PanelOLS(y, X, entity_effects=True, time_effects=True)

# fit the model and print the summary statistics
fixed_effects_results = fixed_effects_model.fit()
fixed_effects_results.summary

C:\Users\alexa\mambaforge\envs\AnalystForecast\Lib\site-packages\linearmodels\panel\model.py:1183: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  super().__init__(dependent, exog, weights=weights, check_rank=check_rank)


Dep. Variable:,Earnings Per Share - Actual Surprise AbsVals,R-squared:,0.0023
Estimator:,PanelOLS,R-squared (Between):,0.0114
No. Observations:,18208,R-squared (Within):,0.0024
Date:,"Sun, Mar 05 2023",R-squared (Overall):,0.0032
Time:,20:53:40,Log-likelihood,-8.858e+04
Cov. Estimator:,Unadjusted,,
,,F-statistic:,8.0485
Entities:,502,P-value,0.0000
Avg Obs:,36.271,Distribution:,"F(5,17663)"
Min Obs:,0.0000,,
Max Obs:,40.000,F-statistic (robust):,7.3448


By Industry: example tech industry

In [18]:
group = df_panel[df_panel['GICS Industry Group Name'] == 'Technology Hardware & Equipment']

group['Instrument'] = group['Instrument'].astype('category')
group['Date'] = pd.to_datetime(group['Date'])
group.set_index(['Instrument', 'Date'], inplace=True)

y = group.loc[:, "Earnings Per Share - Actual Surprise AbsVals"]
X = group.loc[:, "Earnings Per Share – Coefficient of Variation":]

model = PanelOLS(y, X, entity_effects=True, time_effects=True).fit()
model

C:\Users\alexa\AppData\Local\Temp\ipykernel_8088\1151135892.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  group['Instrument'] = group['Instrument'].astype('category')
C:\Users\alexa\AppData\Local\Temp\ipykernel_8088\1151135892.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  group['Date'] = pd.to_datetime(group['Date'])
C:\Users\alexa\mambaforge\envs\AnalystForecast\Lib\site-packages\linearmodels\panel\model.py:1183: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing ob

Dep. Variable:,Earnings Per Share - Actual Surprise AbsVals,R-squared:,0.2022
Estimator:,PanelOLS,R-squared (Between):,-0.3072
No. Observations:,603,R-squared (Within):,0.2098
Date:,"Sun, Mar 05 2023",R-squared (Overall):,0.1601
Time:,20:53:40,Log-likelihood,-2129.1
Cov. Estimator:,Unadjusted,,
,,F-statistic:,27.375
Entities:,19,P-value,0.0000
Avg Obs:,31.737,Distribution:,"F(5,540)"
Min Obs:,1.0000,,
Max Obs:,40.000,F-statistic (robust):,27.045


# COVID Tests

In [25]:
df_clean = df_panel.copy()
df_clean

,Instrument,Date,GICS Industry Group Name,Earnings Per Share - Actual Surprise,Earnings Per Share - Actual Surprise AbsVals,Earnings Per Share – Coefficient of Variation,Number of Analysts,Recommendation - Mean (1-5),Recommendation change,Recommendation - Mean (1-5).1,Price Target - Standard Deviation
0,AVY.N,2013-01-01,Materials,11.178,11.178,0.385569,0.125000,0.742857,0.538462,0.666667,0.002079
1,AVY.N,2013-04-01,Materials,2.482,2.482,0.384977,0.125000,0.696428,0.480770,0.666667,0.003330
2,AVY.N,2013-07-01,Materials,1.068,1.068,0.384792,0.145833,0.696428,0.538462,0.625000,0.003658
3,AVY.N,2013-10-01,Materials,8.095,8.095,0.385251,0.125000,0.603571,0.423078,0.625000,0.002930
4,AVY.N,2014-01-01,Materials,1.471,1.471,0.385411,0.145833,0.495236,0.403846,0.541667,0.002908
...,...,...,...,...,...,...,...,...,...,...,...
19196,POOL.OQ,2021-10-01,Retailing,17.194,17.194,0.385273,0.145833,0.482857,0.538462,0.433333,0.042855
19197,POOL.OQ,2022-01-01,Retailing,40.267,40.267,0.385436,0.145833,0.482857,0.538462,0.433333,0.035088
19198,POOL.OQ,2022-04-01,Retailing,34.342,34.342,0.386695,0.166667,0.371428,0.400001,0.433333,0.031824
19199,POOL.OQ,2022-07-01,Retailing,1.503,1.503,0.385569,0.166667,0.445714,0.630770,0.333333,0.048869


In [21]:
covid_start = pd.to_datetime("2020-01-01")

df_covid = df_clean[df_clean["Date"] >= covid_start]

df_nocovid = df_clean[df_clean["Date"] < covid_start]

In [22]:
df_covid['Instrument'] = df_covid['Instrument'].astype('category')
df_covid['Date'] = pd.to_datetime(df_covid['Date'])

# set the index to be the time variable and the cross-sectional variable
df_covid.set_index(['Instrument', 'Date'], inplace=True)

C:\Users\alexa\AppData\Local\Temp\ipykernel_8088\2704597447.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_covid['Instrument'] = df_covid['Instrument'].astype('category')
C:\Users\alexa\AppData\Local\Temp\ipykernel_8088\2704597447.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_covid['Date'] = pd.to_datetime(df_covid['Date'])


In [23]:
df_nocovid['Instrument'] = df_nocovid['Instrument'].astype('category')
df_nocovid['Date'] = pd.to_datetime(df_nocovid['Date'])

# set the index to be the time variable and the cross-sectional variable
df_nocovid.set_index(['Instrument', 'Date'], inplace=True)

C:\Users\alexa\AppData\Local\Temp\ipykernel_8088\876088522.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_nocovid['Instrument'] = df_nocovid['Instrument'].astype('category')
C:\Users\alexa\AppData\Local\Temp\ipykernel_8088\876088522.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_nocovid['Date'] = pd.to_datetime(df_nocovid['Date'])


## COVID

In [26]:
y = df_covid.loc[:, "Earnings Per Share - Actual Surprise AbsVals"]
X = df_covid.loc[:, "Earnings Per Share – Coefficient of Variation":]

# perform the fixed effects panel regression
fixed_effects_model = PanelOLS(y, X, entity_effects=True, time_effects=True)

# fit the model and print the summary statistics
fixed_effects_results = fixed_effects_model.fit()
fixed_effects_results.summary


C:\Users\alexa\mambaforge\envs\AnalystForecast\Lib\site-packages\linearmodels\panel\model.py:1183: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  super().__init__(dependent, exog, weights=weights, check_rank=check_rank)


Dep. Variable:,Earnings Per Share - Actual Surprise AbsVals,R-squared:,0.0017
Estimator:,PanelOLS,R-squared (Between):,0.0127
No. Observations:,5450,R-squared (Within):,0.0013
Date:,"Sun, Mar 05 2023",R-squared (Overall):,0.0024
Time:,20:55:19,Log-likelihood,-2.704e+04
Cov. Estimator:,Unadjusted,,
,,F-statistic:,1.6678
Entities:,501,P-value,0.1387
Avg Obs:,10.878,Distribution:,"F(5,4934)"
Min Obs:,0.0000,,
Max Obs:,12.000,F-statistic (robust):,1.1238


In [27]:
y = df_nocovid.loc[:, "Earnings Per Share - Actual Surprise AbsVals"]
X = df_nocovid.loc[:, "Earnings Per Share – Coefficient of Variation":]

# perform the fixed effects panel regression
fixed_effects_model = PanelOLS(y, X, entity_effects=True, time_effects=True)

# fit the model and print the summary statistics
fixed_effects_results = fixed_effects_model.fit()
fixed_effects_results.summary


C:\Users\alexa\mambaforge\envs\AnalystForecast\Lib\site-packages\linearmodels\panel\model.py:1183: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  super().__init__(dependent, exog, weights=weights, check_rank=check_rank)


Dep. Variable:,Earnings Per Share - Actual Surprise AbsVals,R-squared:,0.0017
Estimator:,PanelOLS,R-squared (Between):,-0.0134
No. Observations:,12758,R-squared (Within):,0.0018
Date:,"Sun, Mar 05 2023",R-squared (Overall):,-0.0026
Time:,20:55:34,Log-likelihood,-6.055e+04
Cov. Estimator:,Unadjusted,,
,,F-statistic:,4.0907
Entities:,498,P-value,0.0010
Avg Obs:,25.618,Distribution:,"F(5,12266)"
Min Obs:,0.0000,,
Max Obs:,28.000,F-statistic (robust):,4.7537


### Based on these results we can conclude that Analyst factors are a bad indicator for prediction